In [15]:
from pathlib import Path
import ollama

EMBEDDING_MODEL = 'hf.co/nomic-ai/nomic-embed-text-v2-moe-GGUF'
LANGUAGE_MODEL = 'hf.co/tensorblock/scb10x_llama3.1-typhoon2-8b-instruct-GGUF'

DATA_DIR = Path('data')

# Diccionario que contiene embeddings por materia
VECTOR_DB = {} 

# Memoria independiente por materia
CHAT_HISTORY = {}

In [ ]:
def chunk_by_words(text, max_words=1000, overlap=200):
    """Divide el texto en chunks con solapamiento."""
    words = text.split()
    step = max(1, max_words - overlap)
    for i in range(0, len(words), step):
        chunk_words = words[i:i+max_words]
        if not chunk_words:
            break
        yield ' '.join(chunk_words)

In [17]:
def load_datasets():
    """Carga los datasets y genera embeddings por materia."""
    for subject_dir in DATA_DIR.iterdir():
        if subject_dir.is_dir():
            subject = subject_dir.name.lower()
            VECTOR_DB[subject] = []
            CHAT_HISTORY[subject] = []  # Inicializa historial vacío
            files = sorted(subject_dir.glob('*.txt'))

            for txt in files:
                with txt.open('r', encoding='utf-8') as f:
                    text = f.read().strip()
                    for chunk in chunk_by_words(text):
                        embedding = ollama.embed(
                            model=EMBEDDING_MODEL, 
                            input=chunk
                        )['embeddings'][0]
                        VECTOR_DB[subject].append((f"[{txt.name}] {chunk}", embedding))

            print(f"Materia '{subject}': cargados {len(VECTOR_DB[subject])} chunks desde {len(files)} archivos.")

In [18]:
def similaridad_coseno(a, b):
    """Calcula la similaridad coseno entre dos embeddings."""
    producto_matriz = sum(x * y for x, y in zip(a, b))
    norma_a = sum(x ** 2 for x in a) ** 0.5
    norma_b = sum(x ** 2 for x in b) ** 0.5
    return producto_matriz / (norma_a * norma_b)

In [19]:
def detect_subject(query, subjects):
    """Clasifica la pregunta y determina la materia más probable."""
    prompt = f"""Eres un asistente que clasifica preguntas por materia.
Materias disponibles: {', '.join(subjects)}

Pregunta: "{query}"

Responde únicamente con el nombre exacto de la materia más relacionada."""
    response = ollama.chat(
        model=LANGUAGE_MODEL,
        messages=[
            {'role': 'system', 'content': prompt},
        ]
    )
    subject = response['message']['content'].strip().lower()
    return subject if subject in subjects else None

In [20]:
def retrieve(query, subject, top_n=8):
    """Busca los chunks más relevantes dentro de la materia."""
    query_embedding = ollama.embed(
        model=EMBEDDING_MODEL, 
        input=query
    )['embeddings'][0]

    similarities = []
    for chunk, embedding in VECTOR_DB[subject]:
        similarity = similaridad_coseno(query_embedding, embedding)
        similarities.append((chunk, similarity))

    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:top_n]

In [21]:
def build_instruction_prompt(retrieved_knowledge, chat_history):
    """Crea el prompt final con contexto y memoria."""
    history_str = "\n".join([f"Usuario: {u}\nAsistente: {a}" for u, a in chat_history])
    context_str = "\n".join([f" - {chunk}" for chunk, _ in retrieved_knowledge])
    return f"""Eres un chatbot servicial. Usa solo las siguientes piezas de contexto para contestar la pregunta.
Si no sabes la respuesta, admite que no lo sabes.
No inventes información.

Contexto relevante:
{context_str}

Historial de conversación:
{history_str}
"""

In [ ]:
load_datasets()

while True:
    input_query = input("Haceme una pregunta (o escribe 'salir' para terminar): ").strip()
    if input_query.lower() == "salir":
        print("Saliendo del chat...")
        break

    # Detectar materia automáticamente
    subjects = list(VECTOR_DB.keys())
    subject = detect_subject(input_query, subjects)

    if not subject:
        print("⚠️ No pude determinar la materia. Intenta ser más específico.")
        continue

    # Recuperar contexto específico de la materia
    retrieved_knowledge = retrieve(input_query, subject)

    # Construir prompt con memoria de ESA materia
    instruction_prompt = build_instruction_prompt(retrieved_knowledge, CHAT_HISTORY[subject])

    # Generar respuesta
    stream = ollama.chat(
        model=LANGUAGE_MODEL,
        messages=[
            {'role': 'system', 'content': instruction_prompt},
            {'role': 'user', 'content': input_query},
        ],
        stream=True,
    )

    print(f"\nRespuesta ({subject.upper()}):")
    respuesta_completa = ""
    for chunk in stream:
        content = chunk['message']['content']
        respuesta_completa += content
        print(content, end='', flush=True)

    # Guardar turno en la memoria específica de la materia
    CHAT_HISTORY[subject].append((input_query, respuesta_completa))
    print("\n" + "-"*60)

Materia 'cplp': cargados 74 chunks desde 3 archivos.
Materia 'redes': cargados 3 chunks desde 1 archivos.

Respuesta (REDES):
Un protocolo de red es el conjunto de reglas que especifican el intercambio de datos u órdenes durante la comunicación entre las entidades que forman parte de una red. Permiten la comunicación y están implementados en las componentes de la red.
------------------------------------------------------------

Respuesta (REDES):
Un protocolo es un conjunto de reglas que especifican cómo deben interactuar los dispositivos dentro de una red, estableciendo cómo se envían y reciben datos entre ellos. Un protocolo puede incluir información sobre la forma en que se codifican los mensajes, el orden en el que se intercambian esos mensajes y las acciones que se deben realizar al recibir un mensaje u otro evento.
------------------------------------------------------------
Saliendo del chat...
